In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#@title Imports
import torch
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.utils import resample
from scipy.stats import spearmanr
from sklearn.metrics import ndcg_score
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, normalize
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import resample
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, SVC
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score
from scipy.stats import sem
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

# Performs one round of 10-fold cross validation
def tfcv(X, y, MODEL):
  y = np.array(y)
  acc, f1, pre, rec, auroc = [], [], [], [], []
  kf = StratifiedKFold(n_splits=5, shuffle=True)
  for train_index, test_index in kf.split(X, y):
    # Get the train/val data for the current fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pca = PCA(n_components=50)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)

    MODEL.fit(X_train, y_train)
    y_pred = MODEL.predict(X_test)

    # Compute metrics for current fold
    fold_acc = accuracy_score(y_test, y_pred)
    fold_f1 = f1_score(y_test, y_pred)
    fold_pre = precision_score(y_test, y_pred)
    fold_rec = recall_score(y_test, y_pred)
    fold_roc = roc_auc_score(y_test, y_pred)
    acc.append(fold_acc)
    f1.append(fold_f1)
    pre.append(fold_pre)
    rec.append(fold_rec)
    auroc.append(fold_roc)

  return np.mean(np.array(acc)), np.mean(np.array(f1)), np.mean(np.array(pre)), np.mean(np.array(rec)), np.mean(np.array(auroc))

# Optimize a hyperparameter
def optimize(X, y, MODEL, PARAM, param_values):
    top_value = param_values[0]
    top_score = 0
    for i in tqdm(param_values):
        currentscore, _, _, _, _ = tfcv(X, y, MODEL(**{PARAM: i}))
        if currentscore > top_score:
            top_score = currentscore
            top_value = i

    final_acc, final_f1, final_pre, final_rec, final_auroc = [], [], [], [], []
    for i in range(5):
        ac, f1, pre, rec, auroc = tfcv(X, y, MODEL(**{PARAM: top_value}))
        final_acc.append(ac)
        final_f1.append(f1)
        final_pre.append(pre)
        final_rec.append(rec)
        final_auroc.append(auroc)
    final_acc = np.array(final_acc)
    final_f1 = np.array(final_f1)
    final_pre = np.array(final_pre)
    final_rec = np.array(final_rec)
    final_auroc = np.array(final_auroc)
    print(f'Accuracy: {np.mean(final_acc)}, +/- {sem(final_acc)}')
    print(f'F1: {np.mean(final_f1)}, +/- {sem(final_f1)}')
    print(f'Precision: {np.mean(final_pre)}, +/- {sem(final_pre)}')
    print(f'Recall: {np.mean(final_rec)}, +/- {sem(final_rec)}')
    print(f'Auroc: {np.mean(final_auroc)}, +/- {sem(final_auroc)}')
    print(f'top_value: {top_value}')
    return np.mean(final_acc), sem(final_acc)

# Load sequences from csv
df = pd.read_csv('./drive/MyDrive/Data/LazBF_sequences.csv')
LazBF_sequences = df['sequences'].tolist()
LazBF_labels = np.array(df['labels'].tolist())

df = pd.read_csv('./drive/MyDrive/Data/LazBF_sample.csv')
LazBF_sample = df['sequences'].tolist()
LazBF_sample_labels = np.array(df['labels'].tolist())

df = pd.read_csv('./drive/MyDrive/Data/LazDEF_sequences.csv')
LazDEF_sequences = df['sequences'].tolist()
LazDEF_labels = np.array(df['labels'].tolist())

df = pd.read_csv('./drive/MyDrive/Data/LazDEF_sample.csv')
LazDEF_sample = df['sequences'].tolist()
LazDEF_sample_labels = np.array(df['labels'].tolist())

# Load Embs
lazbf_mlm_none = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_none.npy")
lazdef_mlm_none = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_none.npy")

lazbf_mlm_pa = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_PA.npy")
lazdef_mlm_pa = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_PA.npy")

lazbf_mlm_lazbf = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazBF.npy")
lazdef_mlm_lazbf = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazBF.npy")

lazbf_mlm_lazdef = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazDEF.npy")
lazdef_mlm_lazdef = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazDEF.npy")

lazbf_mlm_lazbcdef = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazBCDEF.npy")
lazdef_mlm_lazbcdef = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazBCDEF.npy")

model_list = [LogisticRegression, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
param_list = ['C', 'n_estimators', 'n_estimators', 'C', 'hidden_layer_sizes']
value_list = [
   [1e-2, 1e-1, 1, 5], # LR
   # [5, 25, 50, 80], # KNN
   [5, 25, 50, 100], # RF
   [5, 25, 50, 100], # Ada
   [1e-2, 1e-1, 1, 5], # SVC
   [50, 100, 200, 500], # MLP
]

def balanced_sample_np(seqs, labels, N, seed):
    labels = np.array(labels)
    indices_0 = np.where(labels == 0)[0]
    indices_1 = np.where(labels == 1)[0]
    min_count = min(len(indices_0), len(indices_1), N // 2)
    sampled_indices_0 = np.random.choice(indices_0, min_count, replace=False)
    sampled_indices_1 = np.random.choice(indices_1, min_count, replace=False)
    sampled_indices = np.concatenate((sampled_indices_0, sampled_indices_1))
    np.random.seed(seed)
    random.seed(seed)
    np.random.shuffle(sampled_indices)
    return sampled_indices

In [11]:
#@title LazBF HighN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 1000, 754643236)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.04it/s]


Accuracy: 0.8779999999999999, +/- 0.0010954451150103433
F1: 0.8808005899295429, +/- 0.0011200359900306958
Precision: 0.8620811862183357, +/- 0.00114724560672565
Recall: 0.9016, +/- 0.0015999999999999736
Auroc: 0.8779999999999999, +/- 0.0010954451150103433
top_value: 0.1
Low-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


Accuracy: 0.8827999999999999, +/- 0.0013928388277183812
F1: 0.8858411793557932, +/- 0.0014630753054394926
Precision: 0.8654513851820956, +/- 0.001120142552778452
Recall: 0.9084, +/- 0.00278567765543683
Auroc: 0.8827999999999999, +/- 0.0013928388277183812
top_value: 0.01
Low-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.97, +/- 0.0005477225575051464
F1: 0.9698617474035623, +/- 0.0005395973404175075
Precision: 0.9736713867663891, +/- 0.000707986251995605
Recall: 0.9663999999999999, +/- 0.000748331477354771
Auroc: 0.97, +/- 0.0005477225575051464
top_value: 500
Low-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:34<00:00,  8.57s/it]


Accuracy: 0.8946, +/- 0.0023151673805580242
F1: 0.8954395191327841, +/- 0.0023720659386718772
Precision: 0.8896311447623859, +/- 0.003611622535941364
Recall: 0.9020000000000001, +/- 0.002190890230020646
Auroc: 0.8946, +/- 0.0023151673805580646
top_value: 500
Low-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:35<00:00,  8.77s/it]


Accuracy: 0.877, +/- 0.001095445115010333
F1: 0.8782727854281378, +/- 0.0011255985071274427
Precision: 0.8693690349660674, +/- 0.0014770359894220634
Recall: 0.8880000000000001, +/- 0.0016733200530681526
Auroc: 0.877, +/- 0.001095445115010333
top_value: 100
[[0.8779999999999999, 0.8423999999999999, 0.837, 0.8790000000000001, 0.8695999999999999], [0.8827999999999999, 0.8545999999999999, 0.8382, 0.8826, 0.8678000000000001], [0.9725999999999999, 0.9738, 0.9728, 0.9712, 0.97], [0.8960000000000001, 0.8575999999999999, 0.8634000000000001, 0.8992000000000001, 0.8946], [0.8879999999999999, 0.8524, 0.8555999999999999, 0.8897999999999999, 0.877]]
[[0.0010954451150103433, 0.0032649655434629113, 0.0050990195135928, 0.0011401754250991388, 0.0015033296378372907], [0.0013928388277183812, 0.0021587033144923155, 0.004066939881532554, 0.0015362291495736847, 0.0027820855486487316], [0.0008717797887081354, 0.00037416573867736773, 0.0006633249580710621, 0.001428285685708561, 0.0005477225575051464], [0.00104

In [16]:
#@title LazBF HighN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 1000, 2)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


Accuracy: 0.9059999999999999, +/- 0.00207364413533279
F1: 0.9073954756805053, +/- 0.0020319384137558465
Precision: 0.8952231530692704, +/- 0.002438886535083817
Recall: 0.9204000000000001, +/- 0.002227105745132031
Auroc: 0.9059999999999999, +/- 0.00207364413533279
top_value: 0.1
Low-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:05<00:00,  1.40s/it]


Accuracy: 0.9061999999999999, +/- 0.0018814887722226829
F1: 0.9079440073925558, +/- 0.0018585964825180324
Precision: 0.8917620285043043, +/- 0.002117815000708595
Recall: 0.9256, +/- 0.0021354156504062964
Auroc: 0.9061999999999999, +/- 0.0018814887722226488
top_value: 0.01
Low-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.9742, +/- 0.0006633249580710637
F1: 0.9740643432674971, +/- 0.0006595945990290826
Precision: 0.9783991135438234, +/- 0.0007092940297970056
Recall: 0.97, +/- 0.001095445115010333
Auroc: 0.9742, +/- 0.0006633249580710637
top_value: 500
Low-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:35<00:00,  8.89s/it]


Accuracy: 0.907, +/- 0.0025298221281347165
F1: 0.9078842693347818, +/- 0.002572893746050342
Precision: 0.8989505158244704, +/- 0.002661244706713367
Recall: 0.9179999999999999, +/- 0.003405877273185254
Auroc: 0.907, +/- 0.002529822128134743
top_value: 500
Low-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:37<00:00,  9.43s/it]


Accuracy: 0.9077999999999999, +/- 0.002596150997149436
F1: 0.9076940043377849, +/- 0.0024817827990201136
Precision: 0.909170638617694, +/- 0.003906901880697609
Recall: 0.9067999999999999, +/- 0.0016248076809272222
Auroc: 0.9077999999999999, +/- 0.002596150997149425
top_value: 200
[[0.9059999999999999, 0.8506, 0.8634000000000001, 0.8884000000000001, 0.8858], [0.9061999999999999, 0.8734, 0.8684000000000001, 0.9057999999999999, 0.9054], [0.9711999999999998, 0.9673999999999999, 0.9587999999999999, 0.9745999999999999, 0.9742], [0.9116, 0.8792000000000002, 0.8794000000000001, 0.9160000000000001, 0.907], [0.9097999999999999, 0.8635999999999999, 0.8594000000000002, 0.9006000000000001, 0.9077999999999999]]
[[0.00207364413533279, 0.001400000000000033, 0.0062657800791282225, 0.002088061301782088, 0.00037416573867740633], [0.0018814887722226829, 0.003264965543462912, 0.00304302481094061, 0.0024372115213907642, 0.0018055470085267767], [0.0007348469228349405, 0.00040000000000000034, 0.00203469899493

In [13]:
#@title LazBF MedN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 500, 0)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.09it/s]


Accuracy: 0.8695999999999999, +/- 0.0021354156504062856
F1: 0.8696256470823721, +/- 0.0024871499812285313
Precision: 0.8697372735092648, +/- 0.00336521581046043
Recall: 0.8712, +/- 0.005276362383309158
Auroc: 0.8695999999999999, +/- 0.0021354156504062856
top_value: 1
Low-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.16it/s]


Accuracy: 0.8656, +/- 0.0035440090293338508
F1: 0.8676956858185327, +/- 0.003766491346373964
Precision: 0.8578352326171711, +/- 0.004889386095721053
Recall: 0.8791999999999998, +/- 0.005276362383309153
Auroc: 0.8656, +/- 0.0035440090293338646
top_value: 0.01
Low-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.9655999999999999, +/- 0.002315167380558047
F1: 0.9653480629540102, +/- 0.002307170544114346
Precision: 0.9720832467675024, +/- 0.002468996118089975
Recall: 0.9591999999999998, +/- 0.0023323807579381526
Auroc: 0.9655999999999999, +/- 0.002315167380558047
top_value: 500
Low-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:19<00:00,  4.91s/it]


Accuracy: 0.8556000000000001, +/- 0.007704544113703302
F1: 0.8567434759612975, +/- 0.007995622330264194
Precision: 0.8544350504228996, +/- 0.006422004468171654
Recall: 0.8615999999999999, +/- 0.009682974749528204
Auroc: 0.8556000000000001, +/- 0.007704544113703303
top_value: 100
Low-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:23<00:00,  5.76s/it]


Accuracy: 0.8735999999999999, +/- 0.003867815921162745
F1: 0.8747572758813614, +/- 0.0036516692631172187
Precision: 0.868152806516215, +/- 0.005569696875453738
Recall: 0.8832000000000001, +/- 0.003200000000000011
Auroc: 0.8735999999999999, +/- 0.003867815921162745
top_value: 200
[[0.8695999999999999, 0.8131999999999999, 0.836, 0.8591999999999999, 0.8592000000000001], [0.8656, 0.8315999999999999, 0.8315999999999999, 0.8655999999999999, 0.858], [0.9704, 0.9644, 0.9624, 0.9632, 0.9655999999999999], [0.8836, 0.8464, 0.8488, 0.8800000000000001, 0.8556000000000001], [0.8792, 0.8364, 0.8404, 0.8703999999999998, 0.8735999999999999]]
[[0.0021354156504062856, 0.00417612260356421, 0.00424264068711927, 0.0023323807579381144, 0.0037735924528226046], [0.0035440090293338508, 0.0038157568056677665, 0.0052687759489278355, 0.0017204650534085061, 0.005366563145999478], [0.000748331477354789, 0.001326649916142161, 0.0028565713714171223, 0.0016248076809271838, 0.002315167380558047], [0.005670978751503135, 

In [8]:
#@title LazBF LowN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 200, 1)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.93it/s]


Accuracy: 0.8700000000000001, +/- 0.009354143466934847
F1: 0.8727418219545873, +/- 0.009762900835015605
Precision: 0.8608484441748405, +/- 0.008786641630604202
Recall: 0.8899999999999999, +/- 0.011401754250991386
Auroc: 0.8699999999999999, +/- 0.009354143466934835
top_value: 1
Low-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  2.18it/s]


Accuracy: 0.89, +/- 0.0068920243760451005
F1: 0.8920922203363615, +/- 0.006386478573034399
Precision: 0.8746721510044972, +/- 0.012121899113899286
Recall: 0.9139999999999999, +/- 0.008124038404635985
Auroc: 0.89, +/- 0.0068920243760451005
top_value: 0.1
Low-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.969, +/- 0.005787918451395114
F1: 0.9685549521082255, +/- 0.005899447232791721
Precision: 0.9801804511278196, +/- 0.00553092232947518
Recall: 0.958, +/- 0.006633249580710839
Auroc: 0.969, +/- 0.005787918451395114
top_value: 200
Low-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:08<00:00,  2.18s/it]


Accuracy: 0.861, +/- 0.009669539802906895
F1: 0.8631975649467727, +/- 0.009378931281250572
Precision: 0.8512529037197231, +/- 0.010723084279995895
Recall: 0.8800000000000001, +/- 0.010000000000000031
Auroc: 0.861, +/- 0.009669539802906895
top_value: 500
Low-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:10<00:00,  2.67s/it]


Accuracy: 0.8699999999999999, +/- 0.0031622776601684085
F1: 0.8745671409180666, +/- 0.0036724927042470655
Precision: 0.8540864611032466, +/- 0.003632626567489437
Recall: 0.9019999999999999, +/- 0.005830951894845296
Auroc: 0.8699999999999999, +/- 0.0031622776601684085
top_value: 500
[[0.8700000000000001, 0.8380000000000001, 0.834, 0.866, 0.859], [0.89, 0.898, 0.8639999999999999, 0.9040000000000001, 0.858], [0.9730000000000001, 0.9800000000000001, 0.9690000000000001, 0.9630000000000001, 0.969], [0.9110000000000001, 0.8800000000000001, 0.876, 0.921, 0.861], [0.8880000000000001, 0.852, 0.865, 0.892, 0.8699999999999999]]
[[0.009354143466934847, 0.0090277350426339, 0.010049875621120889, 0.005567764362830031, 0.006000000000000024], [0.0068920243760451005, 0.002549509756796386, 0.002915475947422645, 0.007648529270389173, 0.004898979485566354], [0.003741657386773921, 0.002236067977499804, 0.002915475947422662, 0.0033911649915626175, 0.005787918451395114], [0.007314369419163908, 0.00790569415042

---

In [23]:
#@title LazDEF LowN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 200, 5)

y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbcdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)
# for model, param, grid in zip(model_list, param_list, value_list):

#   # Lazdef prediction
#   print(f"Low-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
#   optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)

#   print(f"Low-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
#   optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)

#   print(f"Low-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
#   optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)

#   print(f"Low-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
#   optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)

Low-N, LazDEF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


Accuracy: 0.767, +/- 0.006633249580710823
F1: 0.7710076583314395, +/- 0.006257862992320949
Precision: 0.7611296091533539, +/- 0.009555103638299763
Recall: 0.7879999999999999, +/- 0.01157583690279022
Auroc: 0.767, +/- 0.006633249580710803
top_value: 1
Low-N, LazDEF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.87it/s]


Accuracy: 0.779, +/- 0.006595452979136473
F1: 0.7812015240222528, +/- 0.008836712740705417
Precision: 0.7742616338254109, +/- 0.003563318980619485
Recall: 0.798, +/- 0.01593737745050924
Auroc: 0.779, +/- 0.006595452979136465
top_value: 0.1
Low-N, LazDEF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.7929999999999999, +/- 0.004636809247747867
F1: 0.7925999251695081, +/- 0.007436808601572019
Precision: 0.7961736361881738, +/- 0.008800917393502458
Recall: 0.796, +/- 0.01964688270438852
Auroc: 0.7929999999999999, +/- 0.004636809247747867
top_value: 200
Low-N, LazDEF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:10<00:00,  2.61s/it]


Accuracy: 0.953, +/- 0.002000000000000035
F1: 0.951203806512282, +/- 0.0021345450768818375
Precision: 0.9753653435301033, +/- 0.007842520138292763
Recall: 0.932, +/- 0.008000000000000004
Auroc: 0.953, +/- 0.002000000000000035
top_value: 500
Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:09<00:00,  2.37s/it]


Accuracy: 0.825, +/- 0.0031622776601683555
F1: 0.8259528333088639, +/- 0.003685554815380428
Precision: 0.8252195343699988, +/- 0.005255851708816717
Recall: 0.8320000000000001, +/- 0.007348469228349525
Auroc: 0.825, +/- 0.0031622776601683555
top_value: 500
[[0.767, 0.681, 0.7130000000000001, 0.7529999999999999, 0.7779999999999999], [0.779, 0.7470000000000001, 0.6979999999999998, 0.7779999999999999, 0.7700000000000001], [0.8280000000000001, 0.744, 0.7310000000000001, 0.8140000000000001, 0.7929999999999999], [0.9710000000000001, 0.951, 0.958, 0.9489999999999998, 0.953], [0.8170000000000002, 0.7219999999999999, 0.7500000000000001, 0.8009999999999999, 0.825]]
[[0.006633249580710823, 0.010653637876331263, 0.015459624833740295, 0.008000000000000009, 0.0033911649915626388], [0.006595452979136473, 0.011895377253370297, 0.008000000000000007, 0.011135528725660069, 0.0070710678118654805], [0.006442049363362577, 0.023473389188610996, 0.008717797887081357, 0.007648529270389165, 0.004636809247747867]

In [18]:
#@title LazDEF MedN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 500, 1)

y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbcdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazDEF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.50it/s]


Accuracy: 0.8084000000000001, +/- 0.0022271057451320173
F1: 0.8106541607600484, +/- 0.0026564954409745906
Precision: 0.8025775640187744, +/- 0.00219796256519915
Recall: 0.8208, +/- 0.005425863986500221
Auroc: 0.8084000000000001, +/- 0.0022271057451320173
top_value: 0.01
Low-N, LazDEF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.65it/s]


Accuracy: 0.8256, +/- 0.0036551333764993986
F1: 0.8298321502739443, +/- 0.0030618013936664175
Precision: 0.812936728292956, +/- 0.004714192507868749
Recall: 0.8503999999999999, +/- 0.0029933259094191526
Auroc: 0.8256, +/- 0.0036551333764993986
top_value: 0.1
Low-N, LazDEF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.8343999999999999, +/- 0.0057061370470748605
F1: 0.8357262143663935, +/- 0.005618605522575245
Precision: 0.830691284307683, +/- 0.006362752364077714
Recall: 0.8432000000000001, +/- 0.005571355310873643
Auroc: 0.8343999999999999, +/- 0.0057061370470748544
top_value: 500
Low-N, LazDEF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:18<00:00,  4.57s/it]


Accuracy: 0.9704, +/- 0.001720465053408532
F1: 0.9700284450515543, +/- 0.0017701317331647952
Precision: 0.9786423168025818, +/- 0.001547947650362011
Recall: 0.9624, +/- 0.003249615361854376
Auroc: 0.9704, +/- 0.001720465053408532
top_value: 100
Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:22<00:00,  5.64s/it]


Accuracy: 0.8383999999999998, +/- 0.005306599664568684
F1: 0.8394006235575672, +/- 0.005668985954637245
Precision: 0.836310261607012, +/- 0.0050920431669243486
Recall: 0.8448, +/- 0.007939773296511674
Auroc: 0.8383999999999998, +/- 0.0053065996645686505
top_value: 50
[[0.8084000000000001, 0.7652, 0.7556, 0.7936, 0.806], [0.8256, 0.778, 0.7487999999999999, 0.8208, 0.8276], [0.8428000000000001, 0.7904000000000001, 0.7836000000000001, 0.8400000000000001, 0.8343999999999999], [0.976, 0.976, 0.9616, 0.9756, 0.9704], [0.8392000000000002, 0.7735999999999998, 0.766, 0.8448, 0.8383999999999998]]
[[0.0022271057451320173, 0.007337574531137656, 0.007756287771866114, 0.006462197768561403, 0.005513619500836073], [0.0036551333764993986, 0.00419523539268062, 0.002799999999999984, 0.005535341001239229, 0.0044452221541785904], [0.006019966777316978, 0.007110555533852492, 0.004707440918375938, 0.007402702209328691, 0.0057061370470748605], [0.0010954451150103736, 0.001264911064067388, 0.002561249694973142

In [19]:
#@title LazDEF HighN
idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 1000, 1)

y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbcdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazDEF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.03it/s]


Accuracy: 0.8026, +/- 0.001503329637837292
F1: 0.8041688854963777, +/- 0.0011139568430028198
Precision: 0.8000366633920054, +/- 0.00214785640005595
Recall: 0.8099999999999999, +/- 0.0008944271909998918
Auroc: 0.8026, +/- 0.0015033296378372898
top_value: 0.1
Low-N, LazDEF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


Accuracy: 0.8106, +/- 0.003108054053583994
F1: 0.811123943122737, +/- 0.0031752698120635576
Precision: 0.8100937753628727, +/- 0.003201394101653392
Recall: 0.8135999999999999, +/- 0.003969886648255866
Auroc: 0.8106, +/- 0.003108054053584024
top_value: 5
Low-N, LazDEF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.8277999999999999, +/- 0.0019849433241279375
F1: 0.8289993268428468, +/- 0.0024721232847361636
Precision: 0.8244714908502144, +/- 0.0016947976999039998
Recall: 0.8348000000000001, +/- 0.005161395160225574
Auroc: 0.8278000000000001, +/- 0.001984943324127919
top_value: 500
Low-N, LazDEF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:31<00:00,  7.87s/it]


Accuracy: 0.9654, +/- 0.0016000000000000061
F1: 0.9650510193511854, +/- 0.0015572874820099422
Precision: 0.9749216610979866, +/- 0.0029671733606685116
Recall: 0.9555999999999999, +/- 0.0009797958971132607
Auroc: 0.9654, +/- 0.0016000000000000187
top_value: 100
Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:41<00:00, 10.47s/it]


Accuracy: 0.807, +/- 0.0029832867780352546
F1: 0.8084860513313525, +/- 0.002840785496555139
Precision: 0.8030939033690256, +/- 0.003654775923320908
Recall: 0.8148000000000002, +/- 0.0037202150475476336
Auroc: 0.807, +/- 0.0029832867780352546
top_value: 200
[[0.8026, 0.7605999999999999, 0.7454, 0.7895999999999999, 0.7874000000000001], [0.8106, 0.7614000000000001, 0.7576, 0.8068, 0.7948000000000001], [0.8311999999999999, 0.7912, 0.7737999999999999, 0.837, 0.8277999999999999], [0.9639999999999999, 0.9650000000000001, 0.9596, 0.9654, 0.9654], [0.8234, 0.7592000000000001, 0.7572, 0.8018000000000001, 0.807]]
[[0.001503329637837292, 0.003414674215792751, 0.0032031234756093804, 0.00305941170815566, 0.0031717503054307056], [0.003108054053583994, 0.0049658836071740575, 0.006087692502089765, 0.002653299832284285, 0.0027820855486486986], [0.0020346989949375635, 0.003929376540877685, 0.004152107898405342, 0.001549193338482925, 0.0019849433241279375], [0.001788854381999855, 0.0008944271909998918, 0.